In [1]:
%load_ext autoreload
%autoreload 2

## Импорт библиотек

In [2]:
import os
import sys

import pandas as pd
import plotly
import plotly.express as px
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, SearchParams, VectorParams
from rich.progress import track
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score

sys.path.append(os.path.pardir)

from project_consts import PROJECT_ROOT

###  Подключаем wandb

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: alica154323 (starminalush). Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(
    # set the wandb project where this run will be logged
    project="intents_classifier",
    name='labse-base-qdrant'
)

##  Константы

In [5]:
FILE_NAME = "intents_chat_bot_pervaya_lin-1000067115-HTA.json"
RAW_DATA_PATH = os.path.join(PROJECT_ROOT, "data", "raw", FILE_NAME)
PROCESSED_FOLDER_PATH = os.path.join(PROJECT_ROOT, "data", "processed")
VALIDATION_DATA_PATH = os.path.join(PROJECT_ROOT, "data", "raw", "sravni_dataset.xlsx")

## Получение данных

In [6]:
intents_df = pd.read_json(os.path.join(PROCESSED_FOLDER_PATH, "intents_prepared.json"))
intents_df.head()

,intent_id,intent_path,phrase
0,24174474,/Пересекающиеся/Продлить полис,продлить полис осаго
1,24174474,/Пересекающиеся/Продлить полис,мне нужно продлить полис
2,24174474,/Пересекающиеся/Продлить полис,нам нужно продлить полис страхования
3,24174474,/Пересекающиеся/Продлить полис,каким образом я могу продлить полис осаго от
4,24174474,/Пересекающиеся/Продлить полис,помогите продлить страховку


### Визуализация

In [7]:
def visualize_dist(df: pd.DataFrame, column_name: str, graph_filename: str) -> None:
    fig = px.histogram(df, y=column_name).update_yaxes(categoryorder="total ascending")
    fig.update_layout(yaxis={"dtick":1},margin={"t":100,"b":100},height=len(df[column_name].unique())*11)
    fig.write_html(graph_filename)
    plotly.offline.plot(fig, filename=graph_filename)

In [8]:
visualize_dist(
    intents_df, "intent_path", os.path.join(PROJECT_ROOT, "reports/intents.html")
)

##  Векторизация

In [9]:
model = SentenceTransformer("sentence-transformers/LaBSE", cache_folder="./hfmodels")

In [10]:
client = QdrantClient(url="http://localhost:6333")
client.recreate_collection(
    collection_name="intents_collection",
    vectors_config=VectorParams(
        size=model.get_sentence_embedding_dimension(), distance=Distance.COSINE
    ),
)

/var/folders/t7/0zcbppxn02b4wfhmpbfg0f2m0000gn/T/ipykernel_2259/3783581583.py:2: DeprecationWarning:

`recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.



True

Заполняем базу векторами

In [11]:
client.upload_points(
    collection_name="intents_collection",
    points=[
        PointStruct(
            id=idx,
            vector=model.encode(row["phrase"]),
            payload={"intent_path": row["intent_path"]},
        )
        for idx, row in track(
            intents_df.iterrows(),
            description="Upload emb to qdrant...",
            total=len(intents_df),
        )
    ],
)

Output()

##  Поиск интентов

In [12]:
validation_df = pd.read_excel(VALIDATION_DATA_PATH)
validation_df.head(5)

,testCase,comment,request,expectedResponse,expectedState,skip,preActions
0,1,/Системные/Сценарии для оператора/Перевод сраз...,Прошу убрать из рассылок любого характера мой ...,NaN,/Сценарии для оператора Перевод сразу,NaN,NaN
1,2,/Кредиты-займы/Почему просроченный платеж,Здравствуйте. Я брал займ в мфо через приложен...,NaN,/ChatWithOperatorMfo,NaN,NaN
2,3,/Сравни ру/Внести изменения/Как изменить сведе...,Добрый день. Я оформила полис ОСАГО на вашем с...,NaN,/Внесение изменений/Полис,NaN,NaN
3,4,/Пересекающиеся/Не пришёл полис,"Два дня назад оплатил осаго, через сравни ру д...",NaN,/Не пришёл полис,NaN,NaN
4,5,/Кредиты-займы/Закрыть кредит,"Здравствуйте, я через вас взял займ в миг кред...",NaN,/ChatWithOperatorMfo,NaN,NaN


In [13]:
validation_df["comment"] = validation_df["comment"].apply(lambda x: x.strip())

In [14]:
visualize_dist(
    validation_df,
    "comment",
    os.path.join(PROJECT_ROOT, "reports/validation_intents.html"),
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
class VectorClassifier:
    def __init__(self, vector_model, vector_store):
        self._vector_model = vector_model
        self._vector_store = vector_store

    def classify(self, text, k=5) -> str:
        embedding = self._vector_model.encode(text)
        # find top 5

        knn_result = self._vector_store.search(
            collection_name="intents_collection",
            query_vector=embedding,
            limit=1,
            search_params=SearchParams(
                exact=True,  # Turns on the exact search mode
            ),
        )[0]

        return knn_result.payload.get("intent_path"), knn_result.score

In [16]:
classifier = VectorClassifier(vector_model=model, vector_store=client)
classifier.classify(validation_df.iloc[0]["request"])

('/Системные/Сценарии для оператора/Перевод сразу/Отписаться от рассылки',
 0.7035544)

In [17]:
validation_df["model_predict"] = validation_df.apply(
    lambda x: classifier.classify(x["request"])[0], axis=1
)

In [18]:
accuracy = accuracy_score(validation_df["comment"], validation_df["model_predict"])
accuracy

0.3569230769230769

## Логирование данных

In [20]:
#логируем метрику
wandb.log({'accuracy': accuracy}

In [22]:
#логируем график
wandb.log({"train_intents": wandb.Html(open(os.path.join(PROJECT_ROOT, 'reports', 'intents.html')))})
wandb.log({"validation_intents": wandb.Html(open(os.path.join(PROJECT_ROOT, 'reports', 'validation_intents.html')))})

In [26]:
#логируем датасет с неправильно распознанными интентами на валидации
error_intents = validation_df[validation_df['comment'] != validation_df['model_predict']]
error_intents.reset_index()
wandb.log({'error_intents': wandb.Table(dataframe=error_intents)})

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
wandb.finish()

accuracy,▁
accuracy,0.35692
